In [1]:
# !/bin/python3
# https://adventofcode.com/2022/day/19

%load_ext lab_black

In [2]:
import math
import re

from utils import read_input

pattern = re.compile("(\d+)")

In [3]:
def parse(data):
    blueprints = {}
    for line in data:
        match = pattern.findall(line)
        blueprints[int(match[0])] = [
            [int(match[1]), 0, 0, 0],
            [int(match[2]), 0, 0, 0],
            [int(match[3]), int(match[4]), 0, 0],
            [int(match[5]), 0, int(match[6]), 0],
        ]
    return blueprints

In [4]:
def get_best_output(blueprint, configs, max_ore, best, max_time, geodes=0):
    if not configs:
        return geodes

    curr_configs = []
    for config in configs:
        robots = config[0]
        time = config[2]
        if best.get(robots, max_time + 1) < time:
            continue

        best[robots] = time
        resources = config[1]

        for robot in get_robot_options(blueprint, robots, max_ore):
            time_until_next_robot = get_time_until_next_robot(
                blueprint, robots, resources, robot
            )
            curr_time = time + time_until_next_robot

            if curr_time >= max_time:
                curr_geodes = resources[3] + (robots[3] * (max_time - time))
                if curr_geodes > geodes:
                    geodes = curr_geodes
                continue

            curr_resources = []
            for curr_robot, count in enumerate(robots):
                curr_resources.append(
                    resources[curr_robot] + (count * time_until_next_robot)
                )

            curr_robots = tuple(
                r + 1 if i == robot else r for i, r in enumerate(robots)
            )

            curr_resources = make_robot(blueprint[robot], curr_resources)

            if best.get(curr_robots, max_time + 1) < curr_time:
                continue

            best[curr_robots] = curr_time

            if curr_resources[3] > geodes:
                geodes = curr_resources[3]

            curr_configs.append((curr_robots, curr_resources, curr_time))

    return get_best_output(
        blueprint, tuple(curr_configs), max_ore, best, max_time, geodes=geodes
    )


def get_robot_options(blueprint, robots, max_ore):
    options = []
    if have_robots_to_make_robot(blueprint, robots, 3):
        options.append(3)
    if robots[2] < blueprint[3][2]:
        if have_robots_to_make_robot(blueprint, robots, 2):
            options.append(2)
    if robots[1] < blueprint[2][1]:
        if have_robots_to_make_robot(blueprint, robots, 1):
            options.append(1)
    if robots[0] < max_ore:
        if have_robots_to_make_robot(blueprint, robots, 0):
            options.append(0)
    return options


def have_robots_to_make_robot(blueprint, robots, robot):
    for resource, amount in enumerate(blueprint[robot]):
        if amount == 0:
            continue

        if robots[resource] == 0:
            return False
    return True


def get_time_until_next_robot(blueprint, robots, resources, robot):
    time = 0
    for resource, amount in enumerate(blueprint[robot]):
        if amount == 0:
            continue

        needed = amount - resources[resource]
        rounds = math.ceil(needed / robots[resource])
        if rounds > time:
            time = rounds
    return time + 1


def make_robot(blueprint, resources):
    for resource, needed in enumerate(blueprint):
        resources[resource] -= needed
    return tuple(resources)

In [5]:
def part_a(blueprints):
    robots = (1, 0, 0, 0)
    resources = (0, 0, 0, 0)

    total = 0
    for id, blueprint in blueprints.items():
        max_ore = max(blueprint, key=lambda b: b[0])[0]
        curr = id * get_best_output(
            blueprint, [(robots, resources, 0)], max_ore, {}, 24
        )
        total += curr
    return total

In [6]:
def part_b(blueprints):
    robots = (1, 0, 0, 0)
    resources = (0, 0, 0, 0)

    total = 1
    for id, blueprint in blueprints.items():
        if id > 3:
            continue
        max_ore = max(blueprint, key=lambda b: b[0])[0]
        total *= get_best_output(blueprint, [(robots, resources, 0)], max_ore, {}, 32)
    return total

In [9]:
blueprints = parse(read_input(parent=__vsc_ipynb_file__, sample="a"))

print("part a:", part_a(blueprints))
# print("part b:", part_b(blueprints))

part a: 33


In [10]:
blueprints = parse(read_input(parent=__vsc_ipynb_file__))

print("part a:", part_a(blueprints))
print("part b:", part_b(blueprints))

part a: 1382
part b: 31740
